Before you turn this problem in, make sure everything runs as expected. 
  1. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and 
  2. Then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below. 

---

In [66]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## P1 :  Functional Programming Finite State Machines.

In this assignment, we understand how to program a finite state machines (FSM). A FSM has the following components:
  - A set of states $Q: \{s_1, \ldots, s_n \}$.
  - A set of input alphabet: $\Sigma: \{ a_1, \ldots, a_m \}$.
  - A transition function from each state, given an input to a new state: $ \delta: Q \times \Sigma \rightarrow Q$ wherein $\delta(s_j, a_k)$ is a state $s_l$ that is reached when the alphabet $a_k$ is encountered in state $s_j$.
  - An initial state $s_0 \in Q$ and 
  - A set of accepting states $F \subseteq Q$.

If you have never encountered an FSM or its definition in the past (ouch!), then please look up these notes online: http://infolab.stanford.edu/~ullman/ialc/spr10/slides/fa2.pdf

Let us implement FSMs using mutables complete the missing functions.

In [67]:
type StateID = Int
type Alph = Int

class MutableFSM(val delta: StateID => Alph => StateID,
          val init_state_id: StateID, 
          val is_accepting: StateID => Boolean){
    
    /*-- Maintain the ID of the current state --*/
    var current_state_id : StateID = init_state_id
    /*-- A getter for the current state --*/
    def get_current_state: StateID = { current_state_id }
    /*-- Is the current state accepting? --*/
    def accepting: Boolean =  { is_accepting(current_state_id) }
    /*-- Apply transition from current state for alphabet `alph`--*/
    def step(alph: Int):Unit ={
        // YOUR CODE HERE
        // find the rule from page 11 basis
       current_state_id = delta(current_state_id )(alph)
        return current_state_id 
    }
    /*-- multiple_steps on a list of alphabets
    Use of for-loops ALLOWED for this one problem!
    --*/
    def multiple_steps(alph_seq: List[Int]): Unit = {
        // YOUR CODE HERE
        for(alph <- alph_seq)
        {
            step(alph)
        }
    }
}

defined type StateID
defined type Alph
defined class MutableFSM

In [68]:
//BEGIN TESTS
def delta (st: StateID) (alph: Alph) = (st, alph) match {
    case (0, 0) => 1
    case (0, 1) => 0
    case (0, 2) => 1
    case (1, 0) => 1
    case (1, 1) => 1
    case (1, 2) => 2
    case (2, 1) => 0
    case (2, 0) => 1
    case (2, 2) => 2
}

def is_accepting(st: StateID) = { st == 2 }

val mfsm = new MutableFSM(delta, 0, is_accepting)

mfsm.step(0)
assert (mfsm.get_current_state == 1, s"A: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 1, s"B: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(2)
assert (mfsm.get_current_state == 2, s"C: expected state: 2, your code returns: ${mfsm.get_current_state}")
mfsm.step(2)
assert (mfsm.get_current_state == 2, s"D: expected state: 2, your code returns: ${mfsm.get_current_state}")
mfsm.step(0)
assert (mfsm.get_current_state == 1, s"E: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 1, s"F: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 1, s"G: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(2)
assert (mfsm.get_current_state == 2, s"H: expected state: 2, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
mfsm.multiple_steps(List(0,1,2,2,0,1,1,2,1))
assert (mfsm.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
passed(5)
//END TESTS


*** Tests Passed (5 points) ***


defined function delta
defined function is_accepting
mfsm: MutableFSM = ammonite.$sess.cmd66$Helper$MutableFSM@1e6f3cbe

Let us now explore a functional style of implementing a FSM.
Note the following changes:
  - `current_state_id` is immutable now.
  - The function `step` returns a new FSM whose current state must be set to the next state from current state and alphabet.
  - Similarly the function `multiple_step` returns a new FSM structure. 
  
 __Restrictions__  For the problem, below you are not allowed to use `var` or loops.  Please use `foldLeft` to implement `multiple_step` function.

In [69]:
class FSM(val current_state_id: StateID,
          val delta: StateID => Alph => StateID,
          val is_accepting: StateID => Boolean) {
    /*-- A getter for the current state --*/
    def get_current_state: Int = current_state_id
    /*-- Is the current state accepting? --*/
    def accepting: Boolean = is_accepting(current_state_id)
    /*-- Apply transition from current state for alphabet `alph`--*/
    def step(alph: Int): FSM= {
        // YOUR CODE HERE
        val current_id=delta(current_state_id)(alph)
        return new FSM(current_id, delta, is_accepting)
    }
    /*-- multiple_steps on a list of alphabets
    Use of var/loops not allowed.
    --*/
    def multiple_steps(alph_seq: List[Alph]):FSM = {
        // YOUR CODE HERE
      alph_seq.foldLeft(new FSM(current_state_id, delta, is_accepting))
        {(acc:FSM,alph:Int)=>step(alph)
       }
    }
}

defined class FSM

In [70]:
//BEGIN TEST
def delta (st: StateID) (alph: Alph) = (st, alph) match {
    case (0, 0) => 1
    case (0, 1) => 0
    case (0, 2) => 1
    case (1, 0) => 1
    case (1, 1) => 1
    case (1, 2) => 2
    case (2, 1) => 0
    case (2, 0) => 1
    case (2, 2) => 2
}

def is_accepting(st: StateID) = { st == 2 }

val mfsm0 = new FSM(0, delta, is_accepting)

val mfsm1 = mfsm0.step(0)
assert (mfsm1.get_current_state == 1, s"A: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm2 = mfsm1.step(1)
assert (mfsm2.get_current_state == 1, s"B: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm3 = mfsm2.step(2)
assert (mfsm3.get_current_state == 2, s"C: expected state: 2, your code returns: ${mfsm.get_current_state}")
val mfsm4 = mfsm3.step(2)
assert (mfsm4.get_current_state == 2, s"D: expected state: 2, your code returns: ${mfsm.get_current_state}")
val mfsm5 = mfsm4.step(0)
assert (mfsm5.get_current_state == 1, s"E: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm6 = mfsm5.step(1)
assert (mfsm6.get_current_state == 1, s"F: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm7 = mfsm6.step(1)
assert (mfsm7.get_current_state == 1, s"G: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm8 = mfsm7.step(2)
assert (mfsm8.get_current_state == 2, s"H: expected state: 2, your code returns: ${mfsm.get_current_state}")
val mfsm9 = mfsm8.step(1)
assert (mfsm9.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
val mfsm10 = mfsm9.multiple_steps(List(0,1,2,2,0,1,1,2,1))
assert (mfsm10.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
passed(10)
//END TEST


*** Tests Passed (10 points) ***


defined function delta
defined function is_accepting
mfsm0: FSM = ammonite.$sess.cmd68$Helper$FSM@15ca9c5d
mfsm1: FSM = ammonite.$sess.cmd68$Helper$FSM@4e3ef386
mfsm2: FSM = ammonite.$sess.cmd68$Helper$FSM@30e704cb
mfsm3: FSM = ammonite.$sess.cmd68$Helper$FSM@76793d3a
mfsm4: FSM = ammonite.$sess.cmd68$Helper$FSM@2813eeda
mfsm5: FSM = ammonite.$sess.cmd68$Helper$FSM@29c680dc
mfsm6: FSM = ammonite.$sess.cmd68$Helper$FSM@43a7f591
mfsm7: FSM = ammonite.$sess.cmd68$Helper$FSM@13349b93
mfsm8: FSM = ammonite.$sess.cmd68$Helper$FSM@488e8e61
mfsm9: FSM = ammonite.$sess.cmd68$Helper$FSM@3d442c32
mfsm10: FSM = ammonite.$sess.cmd68$Helper$FSM@6e528401

# Problem 2: Block of Statements in Lettuce with References (20 points).

You have been using a sequence of statements in almost all languages that you have learned but for Lettuce.  If we write a sequence of expressions (statements) as follows

~~~
<expression 1> ;
<expression 2> ;
<expression 3> ;
<expression 4> ;
... 
<expression n>
~~~
Each of these expressions is interpreted in sequence starting from 1 to n. The return value of the entire
block of expressions (statements) is simply the value returned by the very last statement. 

We would like to add this to Lettuce in the form of a new production.

$$\mathbf{Expr} \ \rightarrow\ Seq(\mathbf{Expr}, \mathbf{Expr})$$

The expression $\texttt{Seq(e1, e2)}$ is a sequence `e1` followed by `e2`. The overall value of a `Seq` is that
returned by `e2`. However, since we have Lettuce with references, it is possible that `e1` may have side effects that are visible in `e2`.

Longer compositions can be obtained by nesting Seq.

~~~
Seq( e1, Seq(e2, Seq(e3, Seq(e4, e5))))
~~~

## 2 A (5 points): Extend the abstract syntax with Seq

In [71]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// New Ref
case class NewRef(e: Expr) extends Expr

//DeRef
case class DeRef(lval: Expr) extends Expr

//AssignRef
case class AssignRef(lval: Expr, rval: Expr) extends Expr

// Seq
// YOUR CODE HERE
case class Seq(e1: Expr, e2: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class Eq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef
defined class Seq

In [72]:
//BEGIN TEST
val s1 = Seq(Const(10.0), NewRef(Const(15.0)))
val s2 = Seq(DeRef(NewRef(Const(20.0))), Let("x", DeRef(NewRef(Const(25.0))), Ident("x")))
val s3 = Seq(Const(10.0), Seq(Const(25.0), Const(35.0)))
val s4 = Seq(Seq(Const(5.0), Const(10.0)),  Seq(Const(25.0), Const(35.0)) )

passed(5)
//END TEST


*** Tests Passed (5 points) ***


s1: Seq = Seq(Const(10.0), NewRef(Const(15.0)))
s2: Seq = Seq(
  DeRef(NewRef(Const(20.0))),
  Let("x", DeRef(NewRef(Const(25.0))), Ident("x"))
)
s3: Seq = Seq(Const(10.0), Seq(Const(25.0), Const(35.0)))
s4: Seq = Seq(Seq(Const(5.0), Const(10.0)), Seq(Const(25.0), Const(35.0)))

## 2 B (5 Points): Write Semantics

We would like to write a semantic rule for evaluating $\texttt{Seq(e1, e2)}$. Complete the rules for the
OK and error cases by filling in the ??? marks below.

$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma, s) = (v_1, s_1),\;\; v_1 \not= \mathbf{error},\;\;\mathbf{eval}(\texttt{e2}, \sigma, \color{red}{1} ) = (v_2, s_2)\\
\hline
\mathbf{eval}(\texttt{Seq(e1, e2)}, \sigma, s) = \color{red}{2} \\
\end{array}(\text{seq-ok})$$


$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma, s) = (v_1, s_1),\;\; v_1 = \mathbf{error} \\
\hline
\mathbf{eval}(\texttt{Seq(e1, e2)}, \sigma, s) = \color{red}{3} \\
\end{array}(\text{seq-nok})$$

Fill in the appropriate values for  $\color{red}{1}, \color{red}{2}, \color{red}{3}$.

Write your answer in the cell below. You can make a numbered list in markdown to represent your answers as follows:
1. First
2. Second
3. And so on...

1. s1
2. (v2,s2)
3. (error,s1)

## 2 C (10 points): Implement the code

Extend the implementation of the interpreter from the notebook on references to add support for Seq.

In [30]:
sealed trait Value
/*-- Now we can finish the rest --*/
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/*-- Note: to get recursion working, we will need to make environments different --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
/* -- references are here -- */
case class Reference(j: Int) extends Value
case object ErrorValue extends Value
/*2. Operators on values */
def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}
def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}
def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

/*3. Immutable Store */
case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }
    
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (NumValue(v3), store2)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x) 
                (env(x), store)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        case NewRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            val (store2, j) = createNewCell(store1, v)
            (Reference(j), store2)
        }
        
        case DeRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            v match {
                case Reference(j) => {
                    val v = lookupCellValue(store1, j)
                    (v, store1)
                }
                case _ => throw new IllegalArgumentException(s"Deref applied to an expression that does not evaluate to a reference")
            }
        }
        
        case AssignRef(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            v1 match {
                case Reference(j) => {
                    val (v2, store2) = evalExpr(e2, env, store1)
                    val store3 = assignToCell(store2, j, v2)
                    (v2, store3)
                }
                case _ => throw new IllegalArgumentException(s"AssignRef applied to argument that is not a reference")
                
            }
        }
        
        case Seq(e1, e2) => {
            // YOUR CODE HERE
             val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
             (v2,store2)
        }
        
    }

}

def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(0, Map()))
            v1
        }
}
    

defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined class Reference
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined class ImmutableStore
defined function createNewCell
defined function lookupCellValue
defined function assignToCell
defined function evalExpr
defined function evalProgram

In [31]:
//BEGIN TEST

val incr = Ident("incr")
val decr = Ident("decr")
val zero = Ident("zero")
val r = Ident("r")
val x = Ident("x")
val e4 = Seq(Seq(Seq(FunCall(zero, r), FunCall(incr, r)), Seq(FunCall(incr, r), FunCall(incr, r))), Seq(FunCall(decr, r), FunCall(decr,r)))
val rr = Let("r", NewRef(Const(15.0)), e4)
val z = Let("zero", FunDef("x", AssignRef(x, Const(0.0))), rr)
val d = Let("decr", FunDef("x", AssignRef(x, Minus(DeRef(x), Const(1.0)))), z)
val i = Let("incr",FunDef("x", AssignRef(x, Plus(DeRef(x), Const(1.0)))), d )
val prog = TopLevel(i)
             
assert(evalProgram(prog) == NumValue(1.0), "Test 1 Set 3 Failed")

passed(5)
//END TEST


*** Tests Passed (5 points) ***


incr: Ident = Ident("incr")
decr: Ident = Ident("decr")
zero: Ident = Ident("zero")
r: Ident = Ident("r")
x: Ident = Ident("x")
e4: Seq = Seq(
  Seq(
    Seq(FunCall(Ident("zero"), Ident("r")), FunCall(Ident("incr"), Ident("r"))),
    Seq(FunCall(Ident("incr"), Ident("r")), FunCall(Ident("incr"), Ident("r")))
  ),
  Seq(FunCall(Ident("decr"), Ident("r")), FunCall(Ident("decr"), Ident("r")))
)
rr: Let = Let(
  "r",
  NewRef(Const(15.0)),
  Seq(
    Seq(
      Seq(
        FunCall(Ident("zero"), Ident("r")),
        FunCall(Ident("incr"), Ident("r"))
      ),
      Seq(
        FunCall(Ident("incr"), Ident("r")),
        FunCall(Ident("incr"), Ident("r"))
      )
    ),
    Seq(FunCall(Ident("decr"), Ident("r")), FunCall(Ident("decr"), Ident("r")))
  )
)
z: Let = Let(
  "zero",
  FunDef("x", AssignRef(Ident("x"), Const(0.0))),
  Let(
    "r",
    NewRef(Const(15.0)),
    Seq(
      Seq(
        Seq(
          FunCall(Ident("zero"), Ident("r")),
          FunCall(Ident("incr"), Ident("r")

## Problem 3  (25 points): Regular Expression Pattern Matching and Continuations

Consider the problem of pattern matching regular expressions. The grammar for regular expressions are given as

$$ \begin{array}{rcl}
\mathbf{RegExpr} & \rightarrow & \text{Atom}(String) \\
& | & \text{Or}(\mathbf{RegExpr}, \mathbf{RegExpr}) \\
& | &  \text{Concat} (\mathbf{RegExpr}, \mathbf{RegExpr}) \\
& | & \text{KleeneStar}( \mathbf{RegExpr} ) \\
& | & \text{And} (\mathbf{RegExpr}, \mathbf{RegExpr}) \\
\end{array}$$


Given a string, $s$ and regular expression $r$, we wish to define a function $\mathbf{Matches}(r, s)$ which returns a tuple $(v_1, j)$
- Wherein $v_1$ is either $true$ if some _prefix_ of the string $s$ matches the expression $r$ or $false$ otherwise. 
- If $v_1 = true$, then $j$ denotes the position where the match ends, such that $j \geq 0$ and $j < \text{length}(s)$. In other words,  the substring $s(0), \ldots, s(j)$ matches the regular expression $r$.
- If $false$, we will simply set $j = -1$.

We will use operational semantics rules to define $\mathbf{Matches}$. 

### Atom

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array} (\text{#3}) }$$

$$\semRule{ s(0,\ldots, j) = t }{ \mathbf{Matches}(\texttt{Atom(t)}, s) = (true, j)}{atom-match}$$ 

$$\semRule{ t\ \text{is not a prefix of}\ s }{ \mathbf{Matches}(\texttt{Atom(t)}, s) = (false, -1)}{atom-no-match}$$ 

### Or

$$\semRule{ \mathbf{Matches}(\texttt{r1}, s) = (v_1, j_1),\ \mathbf{Matches}(\texttt{r2}, s) = (v_2, j_2) }{ \mathbf{Matches}(\texttt{Or(r1, r2)}, s) = (v_1\ \textit{or}\ v_2, \max(j_1, j_2))}{or-match}$$ 

### Concat

$$\semRule{ n = \textsf{length}(s),\ \mathbf{Matches}(\texttt{r1}, s) = (true, j_1), \mathbf{Matches}(\texttt{r2}, s(j_1+1,\ldots, n)) = (true, j_2) }{ \mathbf{Matches}(\texttt{Concat(r1, r2)}, s) = (true, j_1+j_2+1)}{concat-match}$$ 

$$\semRule{ \mathbf{Matches}(\texttt{r1}, s) = (false, -1)  }{ \mathbf{Matches}(\texttt{Concat(r1, r2)}, s) = (false, -1)}{concat-no-match-1}$$ 

$$\semRule{ \mathbf{Matches}(\texttt{r1}, s) = (true, j_1),\ \ \mathbf{Matches}(\texttt{r2}, s) = (false, -1) }{ \mathbf{Matches}(\texttt{Concat(r1, r2)}, s) = (false, -1)}{concat-no-match-2}$$ 

### Kleene Star

$$\semRule{ \mathbf{Matches}(\texttt{r}, s) = (true, j_1),\ n = \text{length}(s),\ \mathbf{Matches}(\texttt{KleeneStar(r)}, s(j+1,\ldots,n) ) = (true, j_2) }{ \mathbf{Matches}(\texttt{KleeneStar(r)}, s) = (true, j_1+j_2+1) }{kleene-star-match}$$ 

$$\semRule{ \mathbf{Matches}(\texttt{r}, s) = (false, -1) }{ \mathbf{Matches}(\texttt{KleeneStar(r)}, s) = (true, -1) }{kleene-star-no-match}$$ 

The Kleene-star-no-match rule looks very strange on first sight. But really, it is trying to say that any string 
matches the Kleene star of a regular expression. However, if the inner regular expression does not match, the Kleene star matches trivially with a match of length 0.


### A (10 points)
Complete the code below for evaluating the semantics above.


In [73]:
sealed trait RegExp
case class Atom(s: String) extends RegExp
case class Or(r1: RegExp, r2: RegExp) extends RegExp
case class Concat(r1: RegExp, r2: RegExp) extends RegExp
case class KleeneStar(r1: RegExp) extends RegExp

def isPrefixOf(t: String, s: String) = {
    /* If string t is a prefix of s, then return true,
       else return false
       */
    s.startsWith(t)
}

def evalRegexp(r: RegExp, s: String ): (Boolean, Int) = r match {
    case Atom(t) => {
        // YOUR CODE HERE
        if(isPrefixOf(t,s))
        {
            (true,t.length-1)
        }
        else
        {
            (false,-1)
        }
    }
    
    case Or(r1, r2) => {
        val (v1, j1) = evalRegexp(r1, s)
        val (v2, j2) = evalRegexp(r2, s)
        ((v1||v2), math.max(j1, j2) )
    }
    
    case Concat(r1, r2) => {
        // YOUR CODE HERE
           val n=s.length
           val (v1, j1) = evalRegexp(r1, s)
        if (v1) {
            val (v2, j2) = evalRegexp(r2, s.substring(j1+1, n))
            (true, j1+j2+1)
        } else {
            (false, -1)
            
        }
    }
    
    case KleeneStar(rHat) => {
        // First match rHat to s
        val (v1, j1) = evalRegexp(rHat, s)
        if (v1) {
            // If match found, recursively match KleeneStar(rHat) to remainder of the string
            val (v2, j2) = evalRegexp(KleeneStar(rHat), s.substring(j1+1, s.length))
            (true, j1+1+j2)
            // Return true and match index
        } else {
            // Otherwise, match is the empty string
            // return true for match succeeding since Kleenestar always succeeds
            // return -1 for the ending index of the match to signal an empty string
            (true, -1)
            
        }
    }
    
    
}

defined trait RegExp
defined class Atom
defined class Or
defined class Concat
defined class KleeneStar
defined function isPrefixOf
defined function evalRegexp

In [74]:
//BEGIN TEST
assert( evalRegexp(Atom("hello"), "helloworld") == (true, 4) , "Test 1 failed")
assert(evalRegexp(Atom("hello"), "Helloworld") == (false, -1), "Test 2 failed")
assert(evalRegexp(Or(Atom("hello"), Atom("Hellow")), "Helloworld") == (true, 5), "Test 3 failed")
passed(4)
//END TEST


*** Tests Passed (4 points) ***


In [75]:
//BEGIN TEST
assert(evalRegexp(KleeneStar(Atom("H")), "Helloworld") == (true, 0), "Test 4 failed")
assert(evalRegexp(Concat(Atom("hell"), Atom("owor")), "helloworld") == (true, 7), "Test 5 failed")
passed(3)
//END TEST


*** Tests Passed (3 points) ***


In [76]:
//BEGIN TEST
val codeRes = evalRegexp(Concat( Or( Atom("what"), Or(Atom("why"), Atom("when") )), KleeneStar(Or(Atom("what"), Atom("why") ))), "whywhywhatwhatwhatwhywhenwhere") 
println(codeRes)
assert(codeRes == (true, 20), "Test 6 failed ")
passed(3)
//END TEST

(true,20)

*** Tests Passed (3 points) ***


codeRes: (Boolean, Int) = (true, 20)

### B (15 points)

Reimplement the interpreter in the CPS so that all recursion happens at the tail position.

In [77]:
def isPrefixOf_k[T](t: String, s: String, k: Boolean => T): T = {
    /* If string t is a prefix of s, then return true,
       else return false
       */
    // YOUR CODE HERE
     k(s.startsWith(t))
}


def evalRegexp_k[T](r: RegExp, s: String, k: (Boolean, Int) => T ): T = r match {
    case Atom(t) => {
        isPrefixOf_k(t, s, v => {
            // YOUR CODE HERE
           if(isPrefixOf(t,s)) {k(true,t.length-1)}
        else {k(false,-1)}

        }) 
    }
    
    case Or(r1, r2) => {
        // YOUR CODE HERE
        val (v1, j1) = evalRegexp(r1, s)
        val (v2, j2) = evalRegexp(r2, s)
        k((v1||v2), math.max(j1, j2) )
    }
    
    case Concat(r1, r2) => {
        // YOUR CODE HERE
         val n=s.length
           val (v1, j1) = evalRegexp(r1, s)
        if (v1) {
            val (v2, j2) = evalRegexp(r2, s.substring(j1+1, n))
            k(true, j1+j2+1)
        } else {
            k(false, -1)
            
        }
    }
    
    case KleeneStar(rHat) => {
        evalRegexp_k(rHat, s, (v1: Boolean, j1: Int) => {
                // YOUR CODE HERE
                  val (v1, j1) = evalRegexp(rHat, s)
        if (v1) {
            val (v2, j2) = evalRegexp(KleeneStar(rHat), s.substring(j1+1, s.length))
            k(true, j1+1+j2)
        } else {
            k(true, -1)
            
        }
        })
        
    }
    
    
}

defined function isPrefixOf_k
defined function evalRegexp_k

In [78]:
//BEGIN TEST
assert( evalRegexp_k[Int](Atom("hello"), "helloworld", (x: Boolean, y: Int) => {y} ) == 4, "Test 1 failed")
assert( evalRegexp_k[Boolean](Atom("hello"), "Helloworld", (x: Boolean, y: Int) => {x} ) == false, "Test 2 failed")
passed(5)
//END TEST


*** Tests Passed (5 points) ***


In [79]:
//BEGIN TEST
assert( evalRegexp_k[String](Or(Atom("hello"), Atom("Hellow")), "Helloworld", (x: Boolean, y: Int) => {y.toString} ) == "5", "Test 2 failed")
passed(5)
//END TEST


*** Tests Passed (5 points) ***


In [80]:
//BEGIN TEST
assert(evalRegexp_k(KleeneStar(Atom("H")), "HHHHHelloworld", (x: Boolean, y: Int) => (x,y)) == (true, 4), "Test 4 failed")
assert(evalRegexp_k(Concat(Atom("hell"), Atom("owor")), "helloworld", (x: Boolean, y: Int) => (y)) == 7, "Test 5 failed")
assert(evalRegexp_k(Concat( Or( Atom("what"), Or(Atom("why"), Atom("when") )), KleeneStar(Or(Atom("what"), Atom("why") ))), "whywhywhatwhatwhatwhywhenwhere", (x: Boolean, y:Int) => (x, y)) == (true, 20), "Test 6 failed")
passed(5)
//END TEST


*** Tests Passed (5 points) ***


## That's All Folks